<a href="https://colab.research.google.com/github/giorginolab/MD-Tutorial-Data/blob/main/etc/pytraj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo ACF in MDTraj

Compares a numpy implementation with AmberTools' cpptraj/pytraj one.

In [1]:
%pip  install -q condacolab   
import condacolab
condacolab.install_miniforge()


✨🍰✨ Everything looks OK!


# Install Python libraries

Should be done via either pip or conda. We may already have them.

In [2]:
!conda install -q mdtraj ambertools=22

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - ambertools=22
    - mdtraj


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ambertools-22.5            |  py310hd182041_0        91.4 MB  conda-forge
    arpack-3.7.0               |       hdefa2d7_2         215 KB  conda-forge
    astunparse-1.6.3           |     pyhd8ed1ab_0          15 KB  conda-forge
    blosc-1.21.4               |       h0f2a231_0          48 KB  conda-forge
    boltons-23.0.0             |     pyhd8ed1ab_0         296 KB  conda-forge
    boost-cpp-1.78.0           |       h6582d0a_3        15.2 MB  conda-forge
    brotli

In [3]:
!cpptraj


CPPTRAJ: Trajectory Analysis. V6.4.4 (AmberTools)
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 05/16/23 21:57:10
| Available memory: 2.341 GB

^C


In [4]:
# Download the test trajectory
!wget -O- https://github.com/Amber-MD/pytraj/archive/refs/tags/v.2.0.5.tar.gz | tar -zxf-

--2023-05-16 21:58:57--  https://github.com/Amber-MD/pytraj/archive/refs/tags/v.2.0.5.tar.gz
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/Amber-MD/pytraj/tar.gz/refs/tags/v.2.0.5 [following]
--2023-05-16 21:58:57--  https://codeload.github.com/Amber-MD/pytraj/tar.gz/refs/tags/v.2.0.5
Resolving codeload.github.com (codeload.github.com)... 140.82.113.10
Connecting to codeload.github.com (codeload.github.com)|140.82.113.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘STDOUT’

-                       [              <=>   ]  36.19M  7.11MB/s    in 4.0s    

2023-05-16 21:59:01 (9.06 MB/s) - written to stdout [37945654]



In [5]:
!find . -name Tc5b.x

./pytraj-v.2.0.5/tests/data/Tc5b.x


# Pytraj

In [6]:
import pytraj as pt

# use iterload for memory saving
traj = pt.iterload("pytraj-v.2.0.5/tests/data/Tc5b.x", "pytraj-v.2.0.5/tests/data/Tc5b.top")

# calculate phi residue 3
dset = pt.calc_phi(traj, resrange='3-7')

# calcuate autocorrelation function for 1st dataset
af = pt.acorr(dset[0])
print(af)

[ 1.         -0.14098525  0.0815446  -0.33501382 -0.3415139   0.12924603
 -0.09997055  0.31578632 -0.112455    0.00336156]


In [7]:
dset

<pytraj.DatasetList with 5 datasets>
phi:3
[-149.03546384  -75.2000377   -78.25317579  -71.02248136  -63.91156199
 -116.60967828  -91.31475319 -132.49270182  -80.4173049   -95.86562288]

phi:4
[-155.79557499 -106.17617884 -134.02131493  -53.55778456  -59.58794832
  -94.98123448  -79.39922554  -64.5076182   -73.3429715   -81.34933091]

phi:5
[-125.82477671 -100.82317865 -118.23384293  -45.42351746  -65.64193036
  -80.093054    -88.08732265  -88.00621383 -108.76069876  -79.56164788]

phi:6
[-154.42303257 -148.64457702 -131.39734601  -82.5877152   -74.32836239
 -110.53379436 -102.63520333 -140.02184247  -68.3960965  -104.7528891 ]

phi:7
[-163.56723225 -146.44236323 -113.67087119 -134.3020574  -108.61681861
 -127.96071037 -122.54726782 -113.74723252 -140.51360719 -137.665533  ]

In [8]:
traj

pytraj.TrajectoryIterator, 10 frames: 
Size: 0.000068 (GB)
<Topology: 304 atoms, 20 residues, 1 mols, non-PBC>
           

# MDTraj

Note that MDTraj relies on file extensions so we symlink the files for loading.

In [14]:
import mdtraj as mdt
import numpy as np

!ln -s pytraj-v.2.0.5/tests/data/Tc5b.top Tc5b.prmtop
!ln -s pytraj-v.2.0.5/tests/data/Tc5b.x Tc5b.mdcrd
traj_mdt = mdt.load("Tc5b.mdcrd", top="Tc5b.prmtop")

ln: failed to create symbolic link 'Tc5b.prmtop': File exists
ln: failed to create symbolic link 'Tc5b.mdcrd': File exists


In [15]:
traj_mdt

<mdtraj.Trajectory with 10 frames, 304 atoms, 20 residues, without unitcells at 0x7fd7626017b0>

In [16]:
phi_i, phi_a = mdt.compute_phi(traj_mdt)

In [17]:
phi_a.shape

(10, 19)

In [18]:
# Residue indices are different for some reason. The results match.
phi_subset = phi_a[:,1:6].T * 180/np.pi
phi_subset

array([[-149.03546 ,  -75.19999 ,  -78.253174, ..., -132.49269 ,
         -80.417305,  -95.865616],
       [-155.79549 , -106.17615 , -134.02132 , ...,  -64.50762 ,
         -73.34293 ,  -81.34929 ],
       [-125.82476 , -100.82319 , -118.23385 , ...,  -88.0062  ,
        -108.76068 ,  -79.561676],
       [-154.42303 , -148.64458 , -131.39734 , ..., -140.02182 ,
         -68.39611 , -104.75288 ],
       [-163.56723 , -146.44235 , -113.670876, ..., -113.74722 ,
        -140.51361 , -137.6655  ]], dtype=float32)

In [19]:
p0=phi_subset[0,:]

In [20]:
# ACF requires a couple of conversions
cor_p0=np.correlate(p0,p0,"full")
cor_p0=cor_p0[cor_p0.size//2:]
cor_p0

array([98266.734, 75752.4  , 71617.37 , 57600.703, 51168.227, 46452.18 ,
       36674.12 , 33295.277, 19194.123, 14287.376], dtype=float32)

In [21]:
# Pytraj/cpptraj use NORMALIZED ACF. 
p0s=(p0-p0.mean())/p0.std()
p0s

array([-1.9940327 ,  0.7516145 ,  0.63807875,  0.90695906,  1.171386  ,
       -0.78824645,  0.15236814, -1.378873  ,  0.55760336, -0.01685786],
      dtype=float32)

In [22]:
# The results match
cor_p0s=np.correlate(p0s,p0s,"full")
cor_p0s=cor_p0s[cor_p0s.size//2:]/len(p0s)
cor_p0s

array([ 1.0000001 , -0.14098492,  0.08154424, -0.3350137 , -0.3415141 ,
        0.12924582, -0.09997027,  0.31578642, -0.112455  ,  0.00336151],
      dtype=float32)